In [1]:
#import pandas as pd

#in_csv = 'conflicts.csv'
#number_lines = sum(1 for row in (open(in_csv)))
#rowsize = 5000
#for i in range(1,number_lines,rowsize):
#    df = pd.read_csv(in_csv,
#          header=None,
#          nrows = rowsize,#number of rows to read at each loop
#          skiprows = i)
#    out_csv = 'conflicts' + str(i) + '.csv'
#    df.to_csv(out_csv,
#          index=False,
#          header=False,
#          mode='a',#append data to csv file
#          chunksize=rowsize)#size of data to append for each loop

import findspark
findspark.init()
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pyspark.sql.types as T

spark = SparkSession \
    .builder \
    .appName("ConflictsApp") \
    .getOrCreate()

In [2]:
part = spark.read.csv(
    "conflicts.csv",
    header=True,
    inferSchema=True,
)
dataSchema = part.schema
df = ( spark \
    .readStream \
    .option("sep", ",") \
    .schema(dataSchema) \
    .option("maxFilesPerTrigger", 1) \
    .csv("data_conflicts/") )  
dataSchema

StructType(List(StructField(_c0,IntegerType,true),StructField(ISO,IntegerType,true),StructField(EVENT_DATE,StringType,true),StructField(EVENT_TYPE,StringType,true),StructField(SUB_EVENT_TYPE,StringType,true),StructField(ACTOR1,StringType,true),StructField(ASSOC_ACTOR_1,StringType,true),StructField(ACTOR2,StringType,true),StructField(ASSOC_ACTOR_2,StringType,true),StructField(INTERACTION,IntegerType,true),StructField(REGION,StringType,true),StructField(COUNTRY,StringType,true),StructField(ADMIN1,StringType,true),StructField(ADMIN2,StringType,true),StructField(LOCATION,StringType,true),StructField(SOURCE,StringType,true),StructField(NOTES,StringType,true),StructField(FATALITIES,StringType,true)))

In [3]:
query = df.groupBy("REGION").count()
query \
    .writeStream \
    .format("memory") \
    .outputMode("complete") \
    .queryName("region") \
    .start()

In [4]:
query.isStreaming

True

In [7]:
for x in range(5):
  spark.sql("select * from region").show()

+---------------+-----+
|         REGION|count|
+---------------+-----+
| Eastern Africa|53597|
| Western Africa|30739|
|Northern Africa|47914|
|Southern Africa|15460|
|  Middle Africa|22868|
+---------------+-----+

+---------------+-----+
|         REGION|count|
+---------------+-----+
| Eastern Africa|53597|
| Western Africa|30739|
|Northern Africa|47914|
|Southern Africa|15460|
|  Middle Africa|22868|
+---------------+-----+

+---------------+-----+
|         REGION|count|
+---------------+-----+
| Eastern Africa|53597|
| Western Africa|30739|
|Northern Africa|47914|
|Southern Africa|15460|
|  Middle Africa|22868|
+---------------+-----+

+---------------+-----+
|         REGION|count|
+---------------+-----+
| Eastern Africa|53597|
| Western Africa|30739|
|Northern Africa|47914|
|Southern Africa|15460|
|  Middle Africa|22868|
+---------------+-----+

+---------------+-----+
|         REGION|count|
+---------------+-----+
| Eastern Africa|53597|
| Western Africa|30739|
|Northern Af